**굵은 텍스트**## Download data and source folder (**Do not Modify**)


In [ ]:
!pip install gdown
import gdown
import zipfile
url = 'https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi'
output = 'source.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile(output, "r") as zip_ref:
    zip_ref.extractall('.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi
To: /content/source.zip
100%|██████████| 11.8M/11.8M [00:00<00:00, 240MB/s]


## Install package for calculating FLOPS (**Do not Modify**)


In [ ]:
!pip install pthflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Necessary Dependencies
- You may import other packages if you want.


In [ ]:
import os
import sys
import os.path as osp

from PIL import Image

import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

import glob
import matplotlib.pyplot as plt

from pthflops import count_ops

# functions that you downloaded from the first code cell.
# please use this code for seed reset, dataloaders, and test function
from src.util import reset
from src.data import get_dataloader
from src.test import test


### 패키지 더 넣을 수 있어요
### src의 코드들 바꿀 순 있는데, 그 경우 그 코드들까지 ㄱ같이 제출.


## Define label names, data directory, device name.


In [ ]:
## DO NOT MODIFY FROM HERE
label_names = ['bug', 'electric', 'fighting', 'fire', 'flying', 'grass', 'ground', 'phychic', 'poison', 'water']
data_dir = 'data'
## DO NOT MODIFY UNTIL HERE

## You may modify device name depending on your workspace spec.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Specify the batch size as you want.
batch_size = 64

## **(TO-DO)** DEFINE YOUR `MyModel`
- Please do not change the class name. Let `MyModel` be your classifier class name.
- Below is just simple example using ResNet18.

In [ ]:
######## Define your classification model.  #######
class MyModel(nn.Module):
    def __init__(self, dim_output=len(label_names)):
        super().__init__()
        self.resnet = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)

        self.resnet.fc = nn.Linear(2048, dim_output)

        ## 리소스 적게 사용할수록 좋아요
    def forward(self, img):
        B, C, H, W = img.shape

        out = self.resnet(img)

        return out
####################################################

## **(Optional)** Make your own loss function.
- You may change `criterion` as you want.
- But make your custom loss function work without changing below lines, which starts from `model.train()`


In [ ]:
def train(model, optimizer, sample):
    ### You may define your own loss function.###
    criterion = nn.CrossEntropyLoss()
    #############################################

    ### make your code work without changing below lines ###
    model.train()

    input = sample['img'].float().to(device)
    label = sample['label'].long().to(device)

    pred = model(input)

    loss = criterion(pred, label)

    num_correct = torch.sum(torch.argmax(pred, dim=-1)==label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    return loss.item(), num_correct.item()
    ##########################################################

## **(TO-DO)** DEFINE YOUR `get_optimizer`
- Please do not change the function name. Let `get_optimizer` be your classifier class name.
- This function should return the optimizer for optimizing model parameters properly


In [ ]:
######## Define your own function for optimizer.  #######
def get_optimizer(model, lr=1e-4, wd=1e-6):
    return optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
##########################################################

## Repeat Training with different 10 random seeds.

*   Do not change `max_epoch` and `num_seeds`.
*   You  may change lines for `MyModel` and `get_optimizer` part if they require additional inputs.


In [ ]:
### Do not change below parameters ###
max_epoch = 5
num_seeds = 10
### Do not change above parameters ###


### Make your code work without changing below lines except for "optimizer" part ###
total_best_accu = []

for seed in range(num_seeds):
    # do not erase this random seed initialization part
    reset(seed)

    # get dataloader by spliting train/test data randomly.
    train_loader, test_loader, train_dataset, test_dataset = get_dataloader(data_dir, label_names, batch_size)

    # you may change below line for model definition if your "MyModel" requires more inputs.
    model = MyModel().to(device)

    # you may change below line for optimizer if your "get optimizer" requires more inputs.
    optimizer = get_optimizer(model)

    ###### do not change below lines. Make your code work with below lines. ####
    best_accu = -100
    for epoch in range(max_epoch):
        avg_tr_loss = 0.0
        avg_tr_correct = 0.0
        for sample in train_loader:
            tr_loss, tr_correct = train(model, optimizer, sample)

            avg_tr_loss += tr_loss / len(train_loader)
            avg_tr_correct += tr_correct / len(train_dataset)

        avg_te_correct = 0.0
        for sample in test_loader:
            te_correct = test(model, sample, device)
            avg_te_correct += te_correct / len(test_dataset)

        best_accu = max(avg_te_correct, best_accu)

    print('<<<<<[SEED {}] BEST ACCU : {}>>>>>'.format(seed, best_accu))
    total_best_accu.append(best_accu)
    if seed < num_seeds-1:
        del model, optimizer
    ############################################################################


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

<<<<<[SEED 0] BEST ACCU : 0.9518072289156627>>>>>
<<<<<[SEED 1] BEST ACCU : 0.9421686746987952>>>>>
<<<<<[SEED 2] BEST ACCU : 0.9421686746987952>>>>>
<<<<<[SEED 3] BEST ACCU : 0.9493975903614459>>>>>
<<<<<[SEED 4] BEST ACCU : 0.9759036144578314>>>>>
<<<<<[SEED 5] BEST ACCU : 0.9518072289156628>>>>>
<<<<<[SEED 6] BEST ACCU : 0.9638554216867471>>>>>
<<<<<[SEED 7] BEST ACCU : 0.9542168674698795>>>>>
<<<<<[SEED 8] BEST ACCU : 0.9566265060240965>>>>>
<<<<<[SEED 9] BEST ACCU : 0.9734939759036145>>>>>


## Calculate your grade **(Do not Modify)**
- Grade is calculated based on your `average best accuracy` and `FLOPS`.
- Higher accuracy and lower flops make your grade better.

In [ ]:
###### do not change below lines #####
num_ops = count_ops(model, torch.rand(1, 3, 112, 112).to(device), verbose=False)

print('Accuracy:', np.mean(total_best_accu))
print('Flops Advantage: ', + 1e8/num_ops[0])

point = np.mean(total_best_accu)+ 1e8/num_ops[0]
print('Total Points : ', point)

threshold = 1.0
max_point = 50

if point > threshold:
    grade = max_point
else:
    grade = np.exp(-(threshold - point)) * max_point

print('YOUR grade is {} point'.format(grade))

Input size: (1, 3, 112, 112)
1,089,618,954 FLOPs or approx. 1.09 GFLOPs
Accuracy: 0.9561445783132532
Flops Advantage:  0.09177520236124674
Total Points :  1.0479197806745
YOUR grade is 50 point
